In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as torchvision_transforms
from torchvision.models import resnet18
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertModel, BertTokenizer
import torch.nn as nn
import torch.optim as optim
import os
from PIL import Image
from tqdm import tqdm

# Model 🤖



In [2]:
class MultiModalModel(nn.Module):
    def __init__(self):
        super(MultiModalModel, self).__init__()
        # Load pre-trained models
        self.resnet = resnet18(pretrained=True)
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Define concatonated layers
        self.multi_modal_layers = nn.Sequential(
            nn.Linear(in_features=self.resnet.fc.out_features + self.bert.config.hidden_size, out_features=512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, image_inputs, text_inputs):
        # Process image input
        image_features = self.resnet(image_inputs)
        image_features = torch.flatten(image_features, 1)  # Flatten the features

        # Process text input
        text_features = self.bert(**text_inputs).last_hidden_state[:, 0, :]  # Get the [CLS] token's features

        # Concatenate features
        combined_features = torch.cat((image_features, text_features), dim=1)

        # Pass through additional layers
        output = self.multi_modal_layers(combined_features)

        output_binary = self.sigmoid(output)

        return output_binary

# Data 📊


In [3]:
class MultiModalDataset(Dataset):
    def __init__(self, dataframe, ai_img_dir, real_img_dir, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.text_idx = dataframe.columns.get_loc('Text')
        self.title_idx = dataframe.columns.get_loc('Title')
        self.image_idx = dataframe.columns.get_loc('Image')
        self.label_idx = dataframe.columns.get_loc('Label')
        # self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.ai_img_dir = ai_img_dir
        self.real_img_dir = real_img_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        text = self.dataframe.iloc[idx, self.text_idx]
        title = self.dataframe.iloc[idx, self.title_idx]
        label = self.dataframe.iloc[idx, self.label_idx]

        img_folder = self.ai_img_dir if label == 1 else self.real_img_dir
        img_name = os.path.join(img_folder, str(self.dataframe.iloc[idx, self.image_idx]))
        image = Image.open(img_name).convert('RGB')

        # only tockenizing text for now
        # text = self.tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=512)

        if self.transform:
            image = self.transform(image)

        return title, text, image, label

In [4]:
class ImageDataset(Dataset):
    """
    create image dataset for loading training images and calculating mean and std of normalization
    for image transforms in the MMM

    input: dataframe with Image, Label columns
    """
    def __init__(self, dataframe, ai_img_dir, real_img_dir, transform=None):
        self.dataframe = dataframe
        self.ai_img_dir = ai_img_dir
        self.real_img_dir = real_img_dir
        self.image_idx = dataframe.columns.get_loc('Image')
        self.label_idx = dataframe.columns.get_loc('Label')
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):

        label = self.dataframe.iloc[idx, self.label_idx]

        img_folder = self.ai_img_dir if label == 1 else self.real_img_dir
        img_name = os.path.join(img_folder, str(self.dataframe.iloc[idx, self.image_idx]))
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

In [5]:
def get_mean_std(loader):
    # Compute the mean and standard deviation of all pixels in the dataset
    num_pixels = 0
    mean = 0.0
    std = 0.0
    for images in loader:
        batch_size, num_channels, height, width = images.shape
        num_pixels += batch_size * height * width
        mean += images.mean(axis=(0, 2, 3)).sum()
        std += images.std(axis=(0, 2, 3)).sum()

    mean /= num_pixels
    std /= num_pixels

    return mean, std

def get_normalization_values(dataframe, ai_img_dir, real_img_dir):

    transforms = torchvision_transforms.Compose([
        torchvision_transforms.ToTensor(),
        torchvision_transforms.Resize((224, 224)),
        torchvision_transforms.CenterCrop(224),
    ])

    dataset = ImageDataset(dataframe, ai_img_dir, real_img_dir, transforms)

    batch_size = 32
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    mean, std = get_mean_std(loader)
    return mean, std

In [6]:
def get_data(ai_dataset_path, real_dataset_path, ai_img_dir, real_img_dir):

    # get the datasets
    ai_data = pd.read_csv(ai_dataset_path)
    real_data = pd.read_csv(real_dataset_path)

    combined_data = pd.concat([ai_data, real_data])

    # shuffle dataset before hand
    combined_data_shuffled = combined_data.sample(frac=1).reset_index(drop=True)

    mean, std = get_normalization_values(combined_data[['Image', 'Label']], ai_img_dir, real_img_dir)

    print(mean, std)

    transform = torchvision_transforms.Compose([
        torchvision_transforms.Resize(256),
        torchvision_transforms.CenterCrop(224),
        torchvision_transforms.ToTensor(),
        torchvision_transforms.Normalize(mean=mean, std=std),
    ])

    # create dataset class
    dataset = MultiModalDataset(
        dataframe=combined_data,
        ai_img_dir=ai_img_dir,
        real_img_dir=real_img_dir,
        transform=transform
    )

    # Split the dataset
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size

    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    return train_dataset, test_dataset

# Training 💪

In [7]:
def train(model, loader, criterion, optimizer, config):

    # Run training and track with wandb
    total_batches = len(loader) * config["epochs"]
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config["epochs"])):
        for title, text, image, label in loader:
            tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
            text = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=512)

            loss = train_batch(text, image, label, model, optimizer, criterion)
            example_ct +=  len(image)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 10) == 0:
                train_log(loss, example_ct, epoch)


def train_batch(text, image, label, model, optimizer, criterion):
    text, image, label = text.to(config["device"]), image.to(config["device"]), label.to(config["device"])

    # Forward pass ➡
    output = model(image, text)
    loss = criterion(torch.squeeze(output), label.float())

    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [8]:
def train_log(loss, example_ct, epoch):
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")

# Testing 🧪

In [9]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for title, text, image, label in test_loader:
            tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
            text = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=512)

            text, image, label = text.to(config["device"]), image.to(config["device"]), label.to(config["device"])
            output = model(image, text)
            _, predicted = torch.max(output.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test images: {correct / total:%}")

    # Save the model
    torch.save(model, "MMM.pt")

# Pipeline 😎

In [10]:
def model_pipeline(config):
    # make the model, data, and optimization problem

    model, train_loader, test_loader, criterion, optimizer = make(config)
    print(model)

    # and use them to train the model
    train(model, train_loader, criterion, optimizer, config)

    # and test its final performance
    test(model, test_loader)

    return model

In [11]:
def make(config):
    # Make the data
    print("getting data...")
    train, test = get_data(
        config["ai_dataset_path"],
        config["real_dataset_path"],
        config["ai_img_dir"],
        config["real_img_dir"],
    )

    print("data collected!")

    train_loader = DataLoader(train, batch_size=config["batch_size"], shuffle=True)
    test_loader = DataLoader(test, batch_size=config["batch_size"], shuffle=True)

    # Make the model
    model = MultiModalModel().to(config["device"])

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

    return model, train_loader, test_loader, criterion, optimizer

In [12]:

config = {
    "epochs": 10,
    "learning_rate": 0.001,
    "batch_size": 32,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "ai_dataset_path": "./newsgpt_dataset.csv",
    "real_dataset_path": "./cnn_dataset.csv",
    "ai_img_dir": "./newsgpt_images",
    "real_img_dir": "./cnn_images",
}

device = torch.device(config["device"])

print("running on " + config["device"])

model_pipeline(config)

running on cuda
getting data...
tensor(8.4699e-07) tensor(5.0835e-07)
data collected!


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in a

MultiModalModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

  0%|          | 0/10 [00:00<?, ?it/s]

Loss after 00288 examples: 0.585


...

 90%|█████████ | 9/10 [02:51<00:19, 19.05s/it]

Loss after 03087 examples: 0.386
Loss after 03407 examples: 0.050


100%|██████████| 10/10 [03:12<00:00, 19.23s/it]


Accuracy of the model on the 86 test images: 53.488372%


MultiModalModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

NameError: name 'model' is not defined